In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
DATA.columns

f = open("column_names_of_data.txt", "a")
for item in DATA.columns:
  f.write(item)
  f.write("\n")

f.close()

In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
DATA = DATA.drop(columns=['timestamp'])
DATA = DATA.astype(float)

In [ ]:
scaler = preprocessing.MinMaxScaler()
names = DATA.columns
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)

In [ ]:
train_data = DATA[:int(DATA.shape[0]*0.8)]
test_data = DATA[int(DATA.shape[0]*0.8):]

In [ ]:
train_data.shape,test_data.shape

((41847, 353), (10462, 353))

In [ ]:
train_data.shape

(41847, 353)

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,40673
1.0,1174


In [ ]:
 train_feat = train_data.drop(columns=['value'])
train_feat = train_feat.to_numpy()
test_feat = test_data.drop(columns=['value'])
test_feat = test_feat.to_numpy()

In [ ]:
train_feat = torch.tensor(train_feat, dtype=torch.float)
test_feat = torch.tensor(test_feat, dtype=torch.float)

In [ ]:
test_labels = test_data['value']
test_labels = test_labels.to_numpy()
test_labels = torch.tensor(test_labels, dtype=torch.float)
test_labels

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_window = 30 #20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq

In [ ]:
train_seq = create_data_seq(train_feat,train_window)

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y = next(iter(train_loader))
print(X)


[tensor([[[0.5250, 0.0000, 0.5566,  ..., 0.5303, 0.3649, 0.8854],
         [0.5250, 0.0000, 0.5566,  ..., 0.3876, 0.3649, 0.7917],
         [0.5250, 0.0000, 0.5566,  ..., 0.4028, 0.3649, 0.9062],
         ...,
         [0.5289, 0.0120, 0.5566,  ..., 0.3187, 0.3514, 0.6979],
         [0.5294, 0.0000, 0.5613,  ..., 0.5455, 0.3243, 0.9271],
         [0.5271, 0.0208, 0.5566,  ..., 0.4750, 0.2703, 0.6354]]])]


In [ ]:
X,y = next(iter(train_loader))
print(X)
print(y)

[tensor([[[0.5199, 0.0130, 0.5472,  ..., 0.2773, 0.3514, 0.5833],
         [0.5194, 0.0165, 0.5472,  ..., 0.1763, 0.3514, 0.4479],
         [0.5184, 0.0191, 0.5472,  ..., 0.2711, 0.3514, 0.6354],
         ...,
         [0.5184, 0.0191, 0.5472,  ..., 0.0880, 0.3649, 0.4688],
         [0.5187, 0.0188, 0.5472,  ..., 0.4300, 0.3514, 0.7708],
         [0.5187, 0.0188, 0.5472,  ..., 0.3962, 0.3649, 0.8646]]])]
tensor([[[0.5179, 0.0189, 0.5472, 0.4835, 0.6633, 0.0000, 0.6889, 0.6327,
          0.6695, 0.0000, 0.6957, 0.6400, 0.6519, 0.0000, 0.6739, 0.6200,
          0.6599, 0.0000, 0.6809, 0.6275, 0.6653, 0.0000, 0.7021, 0.6346,
          0.6488, 0.0000, 0.6739, 0.6200, 0.6818, 0.0000, 0.7083, 0.6538,
          0.6828, 0.0000, 0.7111, 0.6400, 0.6725, 0.1733, 0.6889, 0.6400,
          0.6828, 0.0000, 0.7111, 0.6400, 0.6574, 0.0000, 0.6889, 0.6200,
          0.6605, 0.0000, 0.6889, 0.6200, 0.6555, 0.0000, 0.6739, 0.6200,
          0.6630, 0.0000, 0.6889, 0.6327, 0.6559, 0.0000, 0.6739, 0.6200,


In [ ]:
test_seq = create_data_seq(test_feat,train_window)

In [ ]:
test_loader = DataLoader(test_seq, batch_size=1, shuffle=False) ## در هر فور ما یه تعدادی بچ داریم که میدیم به دل با بچ بیشتر میتونیم بگیم مثلا 2 تا 2 تا بهش بده
X,y = next(iter(test_loader))

In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=352,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
############### mine
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=352,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=352,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
### mine
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=352,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 352)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 352)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8,16)
        self.fc2 = torch.nn.Linear(16,352)


    def forward(self, x):
        x = self.encoder(x)
        #print('LSTM out', x.shape)
        x = self.fc1(x)
        #print('DENSE 1', x.shape)
        x = self.fc2(x)
        #print('DENSE 2', x.shape)

        return x

In [ ]:
#### mine
# class RUAD(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.encoder = encoder().to(device)
#         self.fc1 = torch.nn.Linear(8,16)
#         self.fc2 = torch.nn.Linear(16,352)


#     def forward(self, x):
#         x = self.encoder(x)
#         #print('LSTM out', x.shape)
#         x = self.fc1(x)
#         #print('DENSE 1', x.shape)
#         x = self.fc2(x)
#         #print('DENSE 2', x.shape)

#         return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(352, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 352, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=352, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=352,

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  ### adam ro bezar va bebin
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
for d in train_loader:
    print(len(d[0]))
    print(d[0][0].size())
    break
    ## batch size,(row count,column count) --> sequence

1
torch.Size([1, 20, 352])


In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(352, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 352, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=352, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=352,

In [ ]:
%%time
def train():
    model.train()
    for d in train_loader:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss)

for epoch in range(6):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')


############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### #################result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result of first ruad with adam optimizer
# Epoch: 01, Loss: 0.0071
# Epoch: 02, Loss: 0.0081
# Epoch: 03, Loss: 0.0027
# CPU times: user 14min 5s, sys: 6.38 s, total: 14min 12s
# Wall time: 14min 20s

########### result add hidden layer 32 to decoder too and add decoder to ruad and remove fully connected layers model with gpu and adam optimizer
# Epoch: 01, Loss: 0.1494
# Epoch: 02, Loss: 0.1621
# Epoch: 03, Loss: 0.1690
# CPU times: user 16min 19s, sys: 17.8 s, total: 16min 37s
# Wall time: 16min 21s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer
# Epoch: 01, Loss: 0.0155
# Epoch: 02, Loss: 0.0039
# Epoch: 03, Loss: 0.0154
# CPU times: user 13min 26s, sys: 8.53 s, total: 13min 35s
# Wall time: 13min 46s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer , epoch 6
# Epoch: 01, Loss: 0.0155
# Epoch: 02, Loss: 0.0039
# Epoch: 03, Loss: 0.0154
# Epoch: 04, Loss: 0.0054
# Epoch: 05, Loss: 0.0114
# Epoch: 06, Loss: 0.0200
# CPU times: user 27min 26s, sys: 8.53 s, total: 27min 35s
# Wall time: 26min 57s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer , epoch 6 , window 10
# Epoch: 01, Loss: 0.0155
# Epoch: 02, Loss: 0.0039
# Epoch: 03, Loss: 0.0154
# Epoch: 04, Loss: 0.0054
# Epoch: 05, Loss: 0.0114
# Epoch: 06, Loss: 0.0200
# CPU times: user 27min 26s, sys: 8.53 s, total: 27min 35s
# Wall time: 26min 57s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer , epoch 6 , window 30
# Epoch: 01, Loss: 0.0156
# Epoch: 02, Loss: 0.0138
# Epoch: 03, Loss: 0.0060
# Epoch: 04, Loss: 0.0160
# Epoch: 05, Loss: 0.0158
# Epoch: 06, Loss: 0.0070
# CPU times: user 26min 59s, sys: 12.6 s, total: 27min 11s
# Wall time: 27min 33s

Epoch: 01, Loss: 0.0156
Epoch: 02, Loss: 0.0138
Epoch: 03, Loss: 0.0060
Epoch: 04, Loss: 0.0160
Epoch: 05, Loss: 0.0158
Epoch: 06, Loss: 0.0070
CPU times: user 26min 59s, sys: 12.6 s, total: 27min 11s
Wall time: 27min 33s


In [ ]:
def test(x):
    model.eval()
    x = x.to(device)
    out = model(x)
    return out

In [ ]:
X, y = next(iter(test_loader))
#print(X.shape)
print(len(X[0][0]))

20


In [ ]:
out = test(X[0])

In [ ]:
for d in test_loader:
    print(d[0][0].size())
    break

torch.Size([1, 20, 352])


In [ ]:
for d in test_loader:
    x = d[0][0].to(device)
    y = d[1].to(device)
    out = model(x)
    break

In [ ]:
from sklearn.metrics import mean_squared_error,roc_auc_score
reconstruction = []
loss = []
#y_true = []
for d in test_loader:
    x = d[0][0]
    y = d[1]
    out = test(x)
    reconstruction.append(out)
    loss.append(mean_squared_error(out[0][train_window-1].detach().cpu().numpy(),y[0][0]))
    #y_true.append(d[1].detach().cpu().numpy())

In [ ]:
test_seq_label = create_data_seq(test_labels,train_window)

In [ ]:
temp_list = []
for i in range(len(test_seq_label)):
    temp = test_seq_label[i][1].type(torch.int64)
    temp = temp.detach().cpu().numpy().tolist()
    temp_list.append(temp)

In [ ]:
y_true = []
for i in range(len(temp_list)):
    y_true.append(temp_list[i][0])

In [ ]:
error_df = pd.DataFrame({'loss': loss,
                        'true_class': y_true})
error_df.describe()

,loss,true_class
count,10432.000000,10432.000000
mean,0.016205,0.013133
std,0.039524,0.113848
min,0.001716,0.000000
25%,0.005554,0.000000
50%,0.008180,0.000000
75%,0.015109,0.000000
max,0.297797,1.000000


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(error_df['true_class'],error_df['loss'])

0.9099321830808663

In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
######0.9044238322407449    result add hidden layer 32 to decoder too and add decoder to ruad and remove fully connected layers model with gpu (this is pretty good)
#0.836908594438955     result of first ruad with adam optimizer --> accuracy is lower than SGD so not good for this one at least
#0.7857892667792901    result add hidden layer 32 to decoder too and add decoder to ruad and remove fully connected layers model with gpu and adam optimizer (not good)
#0.9078464497072855    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer (gets better)
#0.9111189026657741    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer (gets better), epoch 6
#0.9078027534134614    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer (gets better), epoch 6 , window 10
#0.9099321830808663    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu and add h,c to each layer (gets better), epoch 6 , window 30